In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd
import numpy as np

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define network name, base dir
network_name = "tki_hooge_raam"
base_dir = Path("..\\..\\ribasim_lumping_data\\")
# Directory D-HYDRO basis calculations
dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")
# Directory D-HYDRO results calculations
dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")
# Directory on where to 
results_dir = Path(base_dir, network_name, "3_ribasim\\network\\")
# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    crs=28992
)
# network.export_to_geopackage()

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = "afwateringseenheden2023_v1_HoogeRaam.shp"
areas_id_column         = "CODE"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type='dhydro',
    set_name='winter', 
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="HoogeRaamRibasim.dsproj_data",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=True,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);
# network.add_basis_network(
#     source_type='dhydro',
#     set_name='winter2', 
#     model_dir=Path(dhydro_basis_dir, "winter"),
#     simulation_name="HoogeRaamRibasim.dsproj_data",
#     dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
#     dhydro_volume_tool_force=True,
#     dhydro_volume_tool_increment=0.1
# );

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))
network.volume_data.bedlevel.ugrid.to_geodataframe().plot(column='bedlevel', cmap='Spectral', legend=True, ax=ax)

In [ ]:
def interp_surface_levels(dataset_volume):
    zlevels = dataset_volume.bedlevel.to_dataframe().T
    zlevels.columns.name = 'node_no'
    zlevels.index = [0]
    bedlevel = zlevels.iloc[[0]]

    surface_df = dataset_volume.surface.to_dataframe().unstack()
    surface_df.replace(0.0, np.nan, inplace=True)
    surface_df = surface_df.T.ffill().T
    surface_df.index.name = 'node_no'
    surface_df = surface_df['surface'].T
    surface_df = pd.concat([bedlevel - 0.01, surface_df]).reset_index(drop=True)
    surface_df.iloc[0] = 0

    for i in range(1, len(surface_df)):
        zlevels = pd.concat([zlevels, bedlevel + dhydro_volume_tool_increment * i])
    zlevels = zlevels.reset_index(drop=True)
    z_range = np.arange(np.floor(zlevels.min().min()), np.ceil(zlevels.max().max())+0.01, dhydro_volume_tool_increment)

    df_out = pd.DataFrame(index=z_range, columns=surface_df.columns)
    df_out.index.name = 'zlevel'
    for col in surface_df.columns:
        df_data_col = pd.DataFrame(index=zlevels[col], data=surface_df[col].values, columns=[col])[col]
        df_out[col] = np.interp(z_range, zlevels[col].values, df_data_col.values)
    return df_out

In [ ]:
surface_df = interp_surface_levels(network.volume_data)
storage_df = (surface_df*0.1).cumsum()

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_data(
    set_name="winter",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1",
                      "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
    simulation_ts=[-1],
);
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_data(
    set_name="zomer",
    model_dir=Path(dhydro_results_dir, "winter"),
    simulation_names=["0_01", "0_02", "0_05", "0_075", "0_1",
                      "0_2", "0_5", "0_75", "1_0", "1_5", "2_0"],
    simulation_ts=[-1],
);

In [ ]:
nodes_h_df = network.map_data.mesh1d_s1.to_dataframe().unstack().mesh1d_s1
old_index = nodes_h_df.index.copy()
nodes_h_df = pd.concat([nodes_h_df[col].sort_values(ignore_index=True) for col in nodes_h_df], axis=1)
nodes_h_df.index = old_index
nodes_h_df.head()

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=False,
    uniweirs=False,
    edges=False,
    structures_ids_to_include=[],
    structures_ids_to_exclude=["S105BBW"],
    edge_ids_to_include=[],
    edge_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.generate_ribasim_network()
# Export to geopackage
network.export_to_geopackage()

In [ ]:
network.boundaries_gdf

In [ ]:
network.split_nodes.head()

In [ ]:
network.basins_gdf.head()

In [ ]:
network.basin_connections_gdf.head()

In [ ]:
network.boundary_connections_gdf.head()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    "weir": "TabulatedRatingCurve",
    "uniweir": "TabulatedRatingCurve",
    "pump": "Pump",
    "culvert": "TabulatedRatingCurve",
    "edge": "ManningResistance",
    "orifice": "TabulatedRatingCurve",
    "boundary_connection": "ManningResistance"
}
# specify translation for specific split_nodes to type of ribasim-node
split_node_id_conversion = {
    "ST303938": "ManningResistance"
}

In [ ]:
# Generate Ribasim model and export to geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
)
ribasim_model.write(Path(results_dir, network.name))